In [0]:
df = spark.read.table("workspace.default.bronze_exports")


In [0]:
from pyspark.sql.functions import when, col
df = df.withColumn(
    "valid_price",
    when((col("product_code") == "AGR") & col("unit_price").between(100, 2000), 1)
    .when((col("product_code") == "TXT") & col("unit_price").between(200, 5000), 1)
    .when((col("product_code") == "ELE") & col("unit_price").between(500, 20000), 1)
    .when((col("product_code") == "AUT") & col("unit_price").between(10000, 80000), 1)
    .when((col("product_code") == "PHA") & col("unit_price").between(20000, 200000), 1)
    .when((col("product_code") == "OIL") & col("unit_price").between(20000, 120000), 1)
    .when((col("product_code") == "MAC") & col("unit_price").between(5000, 50000), 1)
    .otherwise(0)
)

df_silver = df.filter(col("valid_price") == 1).drop("valid_price")


In [0]:
from pyspark.sql.functions import initcap
df_silver = df_silver.select(
    "export_id",
    "country_code",
    initcap("country_name").alias("country_name"),
    initcap("region").alias("region"),
    "product_code",
    initcap("product_name").alias("product_name"),
    "year",
    "export_quantity",
    "export_value_usd",
    "unit_price"
)


In [0]:
spark.sql("DROP TABLE IF EXISTS workspace.default.silver_exports")

df_silver.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("workspace.default.silver_exports")
